In [182]:
import pandas as pd
import pickle
import json
import numpy as np
import os
import dask.dataframe as dd
import csv

In [2]:
def get_siretdf_from_original_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Utilisation d'un dataframe intermediaire pour traiter les Siret unique

    Retour:
        - pd.DataFrame
    """

    dfSIRET = pd.DataFrame.copy(df[['idTitulaires', 'typeIdentifiant', 'denominationSociale']])
    dfSIRET = dfSIRET.drop_duplicates(subset=['idTitulaires'], keep='first')
    dfSIRET.reset_index(inplace=True, drop=True)
    dfSIRET.idTitulaires = dfSIRET.idTitulaires.astype(str)

    dfSIRET["idTitulaires"] = np.where(~dfSIRET["idTitulaires"].str.isdigit(), '00000000000000', dfSIRET.idTitulaires)

    dfSIRET.reset_index(inplace=True, drop=True)

    dfSIRET.rename(columns={
        "idTitulaires": "siret",
        "typeIdentifiant": "siren"}, inplace=True)
    dfSIRET.siren = dfSIRET.siret.str[:9] # 9 = taille du Siren
    dfSIRET.denominationSociale = dfSIRET.denominationSociale.astype(str)

    return dfSIRET
def getArchiveErrorSIRET() -> pd.DataFrame:
    """
    Récupération des siret erronés

    Retour:
        - pd.DataFrame
    """
    archiveErrorSIRET = pd.DataFrame(columns=['siret', 'siren', 'denominationSociale'])
    return archiveErrorSIRET
def get_enrichissement_insee(dfSIRET: pd.DataFrame, path_to_data: str) -> list:
    """
    Ajout des informations Adresse/Activité des entreprises via la base siren Insee

    Retour:
        - list:
            - list[0]: pd.DataFrame -- données principales
            - list[1]: pd.DataFrame -- données ou le SIRET n'est pas renseigné
    """
    # dans StockEtablissement_utf8, il y a principalement : siren, siret, nom établissement, adresse, activité principale
    path = os.path.join(path_to_data, conf_data["base_sirene_insee"])
    columns = [
        'siren',
        'nic',
        'siret',
        'typeVoieEtablissement',
        'libelleVoieEtablissement',
        'codePostalEtablissement',
        'libelleCommuneEtablissement',
        'codeCommuneEtablissement',
        'activitePrincipaleEtablissement',
        'nomenclatureActivitePrincipaleEtablissement']  # Colonne à utiliser dans la base Siren
    dtypes = {
        'siret': 'string',
        'typeVoieEtablissement': 'string',
        'libelleVoieEtablissement': 'string',
        'codePostalEtablissement': 'string',
        'libelleCommuneEtablissement': 'string',
    }

    result = pd.DataFrame(columns=columns)
    chunksize = 1000000
    for gm_chunk in pd.read_csv(path, chunksize=chunksize, sep=',', encoding='utf-8', usecols=columns, dtype=dtypes):
        resultTemp = pd.merge(dfSIRET['siret'], gm_chunk, on=['siret'], copy=False)
        result = pd.concat([result, resultTemp], axis=0, copy=False)
        del resultTemp
    result = result.drop_duplicates(subset=['siret'], keep='first')

    enrichissement_insee_siret = pd.merge(dfSIRET, result, how='outer', on=['siret'], copy=False)
    enrichissement_insee_siret.rename(columns={"siren_x": "siren"}, inplace=True)
    enrichissement_insee_siret.drop(columns=["siren_y"], axis=1, inplace=True)
    nanSiret = enrichissement_insee_siret[enrichissement_insee_siret.activitePrincipaleEtablissement.isnull()]
    enrichissement_insee_siret = enrichissement_insee_siret[
        enrichissement_insee_siret.activitePrincipaleEtablissement.notnull()]
    nanSiret = nanSiret.loc[:, ["siret", "siren", "denominationSociale"]]

    # Concaténation des deux resultats
    enrichissementInsee = enrichissement_insee_siret

    temp_df = pd.merge(nanSiret, result, indicator=True, how="outer", on='siren', copy=False)
    del result
    nanSiret = temp_df[temp_df['activitePrincipaleEtablissement'].isnull()]
    nanSiret = nanSiret.iloc[:, :3]
    nanSiret.reset_index(inplace=True, drop=True)

    return [enrichissementInsee, nanSiret]


In [3]:
with open('df_nettoye', 'rb') as df_nettoye:
    df = pickle.load(df_nettoye)

In [4]:
dfSIRET = get_siretdf_from_original_data(df)
archiveErrorSIRET = getArchiveErrorSIRET()

### Vestige de code avec DaskDataFrames, que j'avais reussi a faire marcher sur l'autre PC avant que ça crash. 

In [60]:
columns = [
    'siren',
    'nic',
    'siret',
    'typeVoieEtablissement',
    'libelleVoieEtablissement',
    'codePostalEtablissement',
    'libelleCommuneEtablissement',
    'codeCommuneEtablissement',
    'activitePrincipaleEtablissement',
    'nomenclatureActivitePrincipaleEtablissement']  # Colonne à utiliser dans la base Siren
dtypes = {
    'siret': 'string',
    'typeVoieEtablissement': 'string',
    'libelleVoieEtablissement': 'string',
    'codePostalEtablissement': 'string',
    'libelleCommuneEtablissement': 'string',
    'codeCommuneEtablissement': 'object',
}
ddcache = dd.read_csv(r"/home/gaspard/Documents/decp-augmente/data/StockEtablissement_utf8.csv", sep=',', encoding='utf-8', usecols=columns, dtype=dtypes)
ddtest = ddcache.map_partitions( find_missing_siret, dfSIRET, dfcache=pd.DataFrame(data={"siret": ["empty_cache"]}))

ValueError: Not all divisions are known, can't align partitions. Please use `set_index` to set the index.. If you don't want the partitions to be aligned, and are calling `map_partitions` directly, pass `align_dataframes=False`.

### FIn du vestige

Est ce append ou concat est le plsu rapide ? <br>
Append semble être un chouilla plus rapide <br>
Même si ça va à l'inverse de ce qui est dit sur internet, mais notre cas d'utilisation est un peu différent. <br>
Mais c'est vraiment négligeable ici

In [132]:
dfcachee = loading_cache(r"/home/gaspard/Documents/decp-augmente/cache/cache_StockEtablissement_utf8.csv")

In [126]:
dfcache = actualiser_cache(dfSIRET, r"/home/gaspard/Documents/decp-augmente/data/StockEtablissement_utf8.csv", dfcache=pd.DataFrame())

/tmp/ipykernel_40848/963475162.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfcache = dfcache.append(gm_chunk.loc[gm_chunk.siret.isin(dfSiret_to_add.siret.tolist())].copy())
/tmp/ipykernel_40848/963475162.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfcache = dfcache.append(gm_chunk.loc[gm_chunk.siret.isin(dfSiret_to_add.siret.tolist())].copy())
/tmp/ipykernel_40848/963475162.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfcache = dfcache.append(gm_chunk.loc[gm_chunk.siret.isin(dfSiret_to_add.siret.tolist())].copy())
/tmp/ipykernel_40848/963475162.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfcache = dfcache.

In [273]:
def split_on_column_match(dfSIRET: pd.DataFrame, dfcache: pd.DataFrame, column: str):
    """
    Arguments
    -------------
    Two dataframes
    
    
    Returns
    -----------
    Two series, the second one is the object in column are matching in both dataframes (so the elements in cache), the first one is the one not matching n column.
    """
    boolean_mask = dfSIRET.loc[:, str(column)].isin(dfcache.loc[:, str(column)].tolist())
    return dfSIRET.loc[~boolean_mask, str(column)].copy(), dfSIRET.loc[boolean_mask, str(column)].copy()

In [282]:
def loading_cache(path_to_cache):
    with open(path_to_cache, 'rb') as df_cache:
        df = pickle.load(df_cache)
    return df

def actualiser_cache(dfSiret_to_add, path_to_db, dfcache, chunksize=1000000):
    """
    Arguments
    ----------
    dfSiret_to_add (Series) avec une unique colonne

    Returns
    --------------
    

    La fonction parcourt la bdd Insee par chunk. Pour chaque chunk on regarde si il y a des correspondances de siret entre dfSiret_to_add et la bdd insee.
    Si il y a un match, on ajoute alors les lignes de la bdd insee au dataframe cache. Sinon c'est que les siret sont à la fois valide, mais non présent dans le cache.
    On les sépare (on retire ceux trouvé de dfSIRET_to_add) pour pouvoir les mettre dans un second cache.
    """

    for gm_chunk in pd.read_csv(path_to_db, chunksize=chunksize, sep=',', encoding='utf-8', usecols=columns, dtype=dtypes):
        # Ajouter à df cache les infos qu'il faut
        matching = gm_chunk.loc[gm_chunk.siret.isin(dfSiret_to_add.tolist())].copy() # La copie du dataframe qui match parmis le chunk en cours
        dfSiret_to_add = dfSiret_to_add[~dfSiret_to_add.isin(matching.siret.tolist())] 
        dfcache = dfcache.append(matching)
    return dfcache, dfSiret_to_add
def write_cache(dfcache, path_to_cache):
    with open(path_to_cache, 'wb') as pathcache:
        pickle.dump(dfcache, pathcache)
        print('cache ecrit')
    return None

In [283]:
def is_luhn_valid(x: int) -> bool:
    """
    Application de la formule de Luhn à un nombre
    Permet la verification du numero SIREN et Siret d'un acheteur/etablissement

    Retour:
        - bool
    """
    try:
        luhn_corr = [0, 2, 4, 6, 8, 1, 3, 5, 7, 9]
        list_number_in_x = [int(i) for i in list(str(x))]
        l2 = [luhn_corr[i] if (index + 1) % 2 == 0 else i for index, i in enumerate(list_number_in_x[::-1])]
        if sum(l2) % 10 == 0:
            return True
        elif str(x)[:9] == "356000000":  # SIREN de la Poste
            if sum(list_number_in_x) % 5 == 0:
                return True
        return False
    except:
        return False

In [465]:
def enrichissement_insee_cache(dfSIRET: pd.DataFrame, path_to_data: str, path_to_cache_bdd: str, path_to_cache_not_inbdd: str) -> list:
    """
    Ajout des informations Adresse/Activité des entreprises via la base siren Insee par un système de double cache.
    Pour bien comprendre la fonction il y a plusieurs cas possibles concernant un SIRET. Il y a le cas où le siret est valide et match avec la bdd insee (on gère ça avec le premier cache)
    Le cas où le siret est invalide ou OOOOOOOO (siret artificiel inscrit en amont dans enrichissement.py) il n'y a aucune chance de trouver ça dans la bdd insee donc.
    Le dernier cas où un siret est valide mais pas présent en bdd, pour ceux-ci on créé un second cache.
    Dans un cache (dfcache) sont stockés les informations en rpovenance de la bdD insee que l'on gère 
    Dans le second cache (list_siret_not_found) sont stockés les siret valides que l'on doit gérer mais qui ne sotn pas dans la bdd insee

    Arguments
    -----------------
    dfSIRET 
    path_to_data chemin vers le fichier csv de la BdD Insee Etablissement_utf8
    path_to_cache_bdd chemin vers le cache du fichier csv de la BdD Insee 'Etablissement_utf8'
    path_to_cache_not_inbdd:  chemin vers le cache des siret non trouvé dans le fichier csv de la BdD Insee Etablissement_utf8


    Returns
    --------------
        - list:
            - list[0]: pd.DataFrame -- données principales
            - list[1]: pd.DataFrame -- données où le SIRET n'est pas renseigné

    """
    columns = [
        'siren',
        'nic',
        'siret',
        'typeVoieEtablissement',
        'libelleVoieEtablissement',
        'codePostalEtablissement',
        'libelleCommuneEtablissement',
        'codeCommuneEtablissement',
        'activitePrincipaleEtablissement',
        'nomenclatureActivitePrincipaleEtablissement']  # Colonne à utiliser dans la base Siren
    dtypes = {
        'siret': 'string',
        'typeVoieEtablissement': 'string',
        'libelleVoieEtablissement': 'string',
        'codePostalEtablissement': 'string',
        'libelleCommuneEtablissement': 'string',
        'codeCommuneEtablissement': 'object',
    }
    # Traitement pour le cache. Si le siret n'est pas valide ou non renseigné, on va aller chercher dans le cache. Or on veut pas ça, donc on le gère en amont du cache.
    # Ceux qui ont un siret non valide on les vire de df SIRET, on les récupèrera plus tard.
    mask_siret_nonvalide = (~dfSIRET.siret.apply(is_luhn_valid)) | (dfSIRET.siret == '00000000000000')
    dfSIRET_siret_nonvalide = dfSIRET[mask_siret_nonvalide]
    dfSIRET = dfSIRET[~mask_siret_nonvalide]
    
    # Traitons les caches maintenant

    #Le second cache des siret valide not found est traité en premier.
    cache_siret_not_found_exist = os.path.isfile(path_to_cache_not_inbdd)
    if cache_siret_not_found_exist:
        list_siret_not_found = loading_cache(path_to_cache_not_inbdd)
    else:
        list_siret_not_found = []
    

    mask_siret_valide_notfound = dfSIRET.siret.isin(list_siret_not_found)
    dfSIRET_valide_notfound = dfSIRET[mask_siret_valide_notfound]
    # On retire les siret valides mais non trouvés lors des précédents passages du df.
    dfSIRET = dfSIRET[~mask_siret_valide_notfound]
    
    cache_exist = os.path.isfile(path_to_cache_bdd)
    if cache_exist:
        print('loading cache')
        dfcache = loading_cache(path_to_cache_bdd)
        # regarder les siret dans le cache, ceux pas dans le cache on va passer à travers la bdd pour les trouver. Ceux qui ne sont pas dans la BdD sont sauvés dans un 2e cache.
        seriesSIRETnotincache, seriesSIRETincache = split_on_column_match(dfSIRET, dfcache, column="siret")
        besoin_actualiser_cache = not(seriesSIRETnotincache.empty)

        if besoin_actualiser_cache:
            print('actualiser_cache')
            # Ceux pas dans le cache, ajouter au cache leur correspondant bddinsee
            dfcache, seriessiret_valide_mais_not_found_in_bdd = actualiser_cache(seriesSIRETnotincache, path_to_data, dfcache)
            dfcache = dfcache.drop_duplicates(subset=['siret'], keep='first')
            #Update cache de la lsite des sirets valides mais non trouvés
            list_siret_not_found += seriessiret_valide_mais_not_found_in_bdd.tolist()
            
            # Actualise les caches
            write_cache(dfcache, path_to_cache_bdd)
            write_cache(list_siret_not_found, path_to_cache_not_inbdd)
            
    else:
        print('création cache')
        #dfSIRET_to_add = split_on_column_match(dfSIRET, pd.DataFrame(data={"siret": ["empty_cache"]}), column="siret")
        # crécupérer le dataframe correspondant au cache
        dfcache, seriessiret_valide_mais_not_found_in_bdd = actualiser_cache(dfSIRET.siret, path_to_data, dfcache=pd.DataFrame())
        dfcache = dfcache.drop_duplicates(subset=['siret'], keep='first')
        # Créer les cache
        write_cache(dfcache, path_to_cache_bdd)
        write_cache(seriessiret_valide_mais_not_found_in_bdd.tolist(), path_to_cache_not_inbdd)
    enrichissement_insee_siret = pd.merge(dfSIRET, dfcache, how='outer', on=['siret'], copy=False)
    enrichissement_insee_siret.rename(columns={"siren_x": "siren"}, inplace=True)
    enrichissement_insee_siret.drop(columns=["siren_y"], axis=1, inplace=True)
    nanSiret = pd.concat([enrichissement_insee_siret[enrichissement_insee_siret.activitePrincipaleEtablissement.isnull()], dfSIRET_siret_nonvalide, dfSIRET_valide_notfound])
    enrichissement_insee_siret = enrichissement_insee_siret[
        enrichissement_insee_siret.activitePrincipaleEtablissement.notnull()]
    nanSiret = nanSiret.loc[:, ["siret", "siren", "denominationSociale"]]

    # Concaténation des deux resultats
    enrichissementInsee = enrichissement_insee_siret
    nanSiret = nanSiret.iloc[:, :3]
    nanSiret.reset_index(inplace=True, drop=True)

    return [enrichissementInsee, nanSiret]
    

In [424]:
def get_enrichissement_insee(dfSIRET: pd.DataFrame, path_to_data: str) -> list:
    """
    Ajout des informations Adresse/Activité des entreprises via la base siren Insee

    Retour:
        - list:
            - list[0]: pd.DataFrame -- données principales
            - list[1]: pd.DataFrame -- données ou le SIRET n'est pas renseigné
    """
    # dans StockEtablissement_utf8, il y a principalement : siren, siret, nom établissement, adresse, activité principale
    path = path_to_data
    columns = [
        'siren',
        'nic',
        'siret',
        'typeVoieEtablissement',
        'libelleVoieEtablissement',
        'codePostalEtablissement',
        'libelleCommuneEtablissement',
        'codeCommuneEtablissement',
        'activitePrincipaleEtablissement',
        'nomenclatureActivitePrincipaleEtablissement']  # Colonne à utiliser dans la base Siren
    dtypes = {
        'siret': 'string',
        'typeVoieEtablissement': 'string',
        'libelleVoieEtablissement': 'string',
        'codePostalEtablissement': 'string',
        'libelleCommuneEtablissement': 'string',
    }

    result = pd.DataFrame(columns=columns)
    chunksize = 1000000
    for gm_chunk in pd.read_csv(path, chunksize=chunksize, sep=',', encoding='utf-8', usecols=columns, dtype=dtypes):
        resultTemp = pd.merge(dfSIRET['siret'], gm_chunk, on=['siret'], copy=False)
        result = pd.concat([result, resultTemp], axis=0, copy=False)
        del resultTemp
    result = result.drop_duplicates(subset=['siret'], keep='first')

    enrichissement_insee_siret = pd.merge(dfSIRET, result, how='outer', on=['siret'], copy=False)
    enrichissement_insee_siret.rename(columns={"siren_x": "siren"}, inplace=True)
    enrichissement_insee_siret.drop(columns=["siren_y"], axis=1, inplace=True)
    nanSiret = enrichissement_insee_siret[enrichissement_insee_siret.activitePrincipaleEtablissement.isnull()]
    print('Premier nanSiret \n', nanSiret)
    enrichissement_insee_siret = enrichissement_insee_siret[
        enrichissement_insee_siret.activitePrincipaleEtablissement.notnull()]
    nanSiret = nanSiret.loc[:, ["siret", "siren", "denominationSociale"]]

    # Concaténation des deux resultats
    enrichissementInsee = enrichissement_insee_siret
    print(enrichissement_insee_siret)
    temp_df = pd.merge(nanSiret, result, indicator=True, how="outer", on='siren', copy=False)
    del result
    nanSiret = temp_df[temp_df['activitePrincipaleEtablissement'].isnull()]
    nanSiret = nanSiret.iloc[:, :3]
    nanSiret.reset_index(inplace=True, drop=True).drop_duplicates(subset=['siret'], keep='first')

    return [enrichissementInsee, nanSiret]

In [414]:
unpack_no_cache_l = get_enrichissement_insee(dfSIRET,r'/home/gaspard/Documents/decp-augmente/data/StockEtablissement_utf8.csv')

/tmp/ipykernel_40848/563557681.py:33: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  for gm_chunk in pd.read_csv(path, chunksize=chunksize, sep=',', encoding='utf-8', usecols=columns, dtype=dtypes):


KeyboardInterrupt: 

In [466]:
unpack_l = enrichissement_insee_cache(dfSIRET, r'/home/gaspard/Documents/decp-augmente/data/StockEtablissement_utf8.csv', r"/home/gaspard/Documents/decp-augmente/cache/cache_StockEtablissement_utf8.csv", r"/home/gaspard/Documents/decp-augmente/cache/cache_NOTIN_StockEtablissement_utf8.csv")

loading cache
                siret      siren denominationSociale
15144  95651314700001  956513147             DECITRE
                siret      siren denominationSociale  nic  \
15144  95651314700001  956513147             DECITRE  NaN   

      typeVoieEtablissement libelleVoieEtablissement codePostalEtablissement  \
15144                  <NA>                     <NA>                    <NA>   

      libelleCommuneEtablissement codeCommuneEtablissement  \
15144                        <NA>                      NaN   

      activitePrincipaleEtablissement  \
15144                             NaN   

      nomenclatureActivitePrincipaleEtablissement  
15144                                         NaN  


# Opti enrichissement_acheteur

In [41]:
with open('df_nettoye', 'rb') as df_nettoye:
    df = pickle.load(df_nettoye)

In [5]:
def enrichissement_acheteur(df: pd.DataFrame) -> pd.DataFrame:
    """
    Enrichissement des données des acheteurs via les codes siret/siren.
    Dans cette fonction du pipeline, on ré-utilise la Bdd insee (et donc on va ré utiliser les caches utilisés pour enrichissement_insee)

    Return:
        - pd.DataFrame
    """
    dfAcheteurId = df['acheteur.id'].to_frame()
    dfAcheteurId.columns = ['siret']
    dfAcheteurId = dfAcheteurId.drop_duplicates(keep='first')
    dfAcheteurId.reset_index(inplace=True, drop=True)
    dfAcheteurId = dfAcheteurId.astype(str)

    # StockEtablissement_utf8 et les caches
    chemin = r"/home/gaspard/Documents/decp-augmente/data/StockEtablissement_utf8.csv"

    # Chargement des caches qui existent forcément. Donc pas de test sur leur existence
    # Colonnes utiles l'enrichissement
    usecols=['siret', 'codePostalEtablissement', 'libelleCommuneEtablissement', 'codeCommuneEtablissement']


    # chemin = 'dataEnrichissement/StockEtablissement_utf8.csv'
    result = pd.DataFrame(columns=['siret', 'codePostalEtablissement',
                                   'libelleCommuneEtablissement', 'codeCommuneEtablissement'])
    for gm_chunk in pd.read_csv(
            chemin, chunksize=1000000, sep=',', encoding='utf-8',
            usecols=['siret', 'codePostalEtablissement', 'libelleCommuneEtablissement', 'codeCommuneEtablissement']):
        gm_chunk['siret'] = gm_chunk['siret'].astype(str)
        resultTemp = pd.merge(dfAcheteurId, gm_chunk, on="siret", copy=False)
        result = pd.concat([result, resultTemp], axis=0, copy=False)


    result = result.drop_duplicates(subset=['siret'], keep='first')
    enrichissementAcheteur = result
    enrichissementAcheteur.columns = ['acheteur.id', 'codePostalAcheteur', 'libelleCommuneAcheteur',
                                      'codeCommuneAcheteur']
    enrichissementAcheteur = enrichissementAcheteur.drop_duplicates(subset=['acheteur.id'], keep='first')

    df = pd.merge(df, enrichissementAcheteur, how='left', on='acheteur.id', copy=False)
    del enrichissementAcheteur
    return df

In [6]:
out = enrichissement_acheteur(df)

/tmp/ipykernel_25995/616499640.py:26: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  for gm_chunk in pd.read_csv(
/tmp/ipykernel_25995/616499640.py:26: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  for gm_chunk in pd.read_csv(
/tmp/ipykernel_25995/616499640.py:26: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  for gm_chunk in pd.read_csv(
/tmp/ipykernel_25995/616499640.py:26: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  for gm_chunk in pd.read_csv(
/tmp/ipykernel_25995/616499640.py:26: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  for gm_chunk in pd.read_csv(
/tmp/ipykernel_25995/616499640.py:26: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  for gm_ch

In [54]:
test = df['acheteur.id'].to_frame()

In [31]:
def enrichissement_acheteur(df: pd.DataFrame) -> pd.DataFrame:
    """
    Enrichissement des données des acheteurs via les codes siret/siren.
    Dans cette fonction du pipeline, on ré-utilise la Bdd insee (et donc on va ré utiliser les caches utilisés pour enrichissement_insee)

    Return:
        - pd.DataFrame
    """
    df = pd.concat([df, pd.DataFrame(data={"siret": ["erzz"]})])
    dfAcheteurId = df['acheteur.id'].to_frame()
    dfAcheteurId.columns = ['siret']
    dfAcheteurId = dfAcheteurId.drop_duplicates(keep='first')
    dfAcheteurId.reset_index(inplace=True, drop=True)
    dfAcheteurId = dfAcheteurId.astype(str)

    # StockEtablissement_utf8 et les caches
    path_to_cache_bdd = r"/home/gaspard/Documents/decp-augmente/cache/cache_StockEtablissement_utf8.csv"
    path_to_cache_not_in_bdd = r"/home/gaspard/Documents/decp-augmente/cache/cache_NOTIN_StockEtablissement_utf8.csv"

    # Chargement des caches qui existent forcément. Donc pas de test sur leur existence
    sirets_not_found = loading_cache(path_to_cache_not_in_bdd)
    dfcache = loading_cache(path_to_cache_bdd)
    
    # Colonnes utiles l'enrichissement
    usecols=['siret', 'codePostalEtablissement', 'libelleCommuneEtablissement', 'codeCommuneEtablissement']


    dfcache_acheteur = dfcache[usecols].copy()
    enrichissementAcheteur =  pd.merge(dfAcheteurId, dfcache_acheteur, on="siret", copy=False)
    enrichissementAcheteur = enrichissementAcheteur.drop_duplicates(subset=['siret'], keep='first')
    enrichissementAcheteur.columns = ['acheteur.id', 'codePostalAcheteur', 'libelleCommuneAcheteur',
                                      'codeCommuneAcheteur']
    enrichissementAcheteur = enrichissementAcheteur.drop_duplicates(subset=['acheteur.id'], keep='first')

    df = pd.merge(df, enrichissementAcheteur, how='left', on='acheteur.id', copy=False)
    return df

In [51]:
usecols = ['acheteur.id', 'codePostalAcheteur', 'libelleCommuneAcheteur',
                                      'codeCommuneAcheteur']
out_cache.loc[2, usecols]

acheteur.id               18006702700029
codePostalAcheteur                  <NA>
libelleCommuneAcheteur              <NA>
codeCommuneAcheteur                  NaN
Name: 2, dtype: object

In [52]:
out.loc[2, usecols]

acheteur.id               18006702700029
codePostalAcheteur               75001.0
libelleCommuneAcheteur           PARIS 1
codeCommuneAcheteur                75101
Name: 2, dtype: object

In [27]:
out_cache = enrichissement_acheteur(df)

In [36]:
out_cache

,procedure,source,uid,nature,codeCPV,dureeMois,datePublicationDonnees,id,_type,formePrix,...,codeRegionExecution,libelle,libelleRegionExecution,anneeNotification,moisNotification,dureeMoisEstimee,dureeMoisCalculee,codePostalAcheteur,libelleCommuneAcheteur,codeCommuneAcheteur
0,Procédure adaptée,marches-publics.info,215002189000112018180800100,Marché,39314000,1,2019-05-21,2018180800100,Marché,Ferme,...,nan,Manche,NaN,2018,06,False,1.0,<NA>,<NA>,NaN
1,Appel d'offres ouvert,data.gouv.fr_pes,22250001900013202130054002,Marché,45210000-2,35,2021-07-12,202130054002,Marché,Révisable,...,nan,Doubs,NaN,2021,06,False,35.0,<NA>,<NA>,NaN
2,Appel d'offres ouvert,data.gouv.fr_aife,1800670270002920212021F1799800,Accord-cadre,30213100,48,2021-10-06,20212021F1799800,Marché,NaN,...,nan,Paris,NaN,2021,10,False,48.0,<NA>,<NA>,NaN
3,Appel d'offres ouvert,data.gouv.fr_aife,7760564670058720202020F1529600,Accord-cadre,9310000,15,2020-10-12,20202020F1529600,Marché,NaN,...,nan,Paris,NaN,2020,10,False,15.0,77420,CHAMPS-SUR-MARNE,77083
4,Appel d'offres ouvert,data.gouv.fr_aife,130015373000172021130017121600,Accord-cadre,98310000,48,2021-12-02,2021130017121600,Marché,Révisable,...,nan,Var,NaN,2021,12,False,48.0,<NA>,<NA>,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11070,Procédure adaptée,data.gouv.fr_pes,212405682000152019LOGEM*LOT403,Marché,45421000-4,24,2020-02-28,2019LOGEM*LOT403,Marché,Ferme et actualisable,...,nan,Dordogne,NaN,2020,01,False,24.0,<NA>,<NA>,NaN
11071,Procédure adaptée,data.gouv.fr_pes,212405682000152019LOGEM*LOT403,Marché,45421000-4,24,2020-02-28,2019LOGEM*LOT403,Marché,Ferme et actualisable,...,nan,Dordogne,NaN,2020,01,False,24.0,<NA>,<NA>,NaN
11072,Appel d'offres ouvert,data.gouv.fr_pes,200053551000122020190000002800,Accord-cadre,39222000-4,11,2020-03-02,2020190000002800,Marché,Ferme,...,84,Haute-Savoie,Auvergne-Rhône-Alpes,2020,01,False,11.0,<NA>,<NA>,NaN
11073,Appel d'offres ouvert,data.gouv.fr_pes,200053551000122020190000002800,Accord-cadre,39222000-4,11,2020-03-02,2020190000002800,Marché,Ferme,...,84,Haute-Savoie,Auvergne-Rhône-Alpes,2020,01,False,11.0,<NA>,<NA>,NaN


In [66]:
df_keys = pd.concat([df.loc[:, "idTitulaires"], df.loc[:, "acheteur.id"]], names='try')

In [72]:
df_keys.to_frame().rename(columns={0: "siret"})

,siret
0,49994465000013
1,38014942700136
2,43210532800020
3,54210765111679
4,43001432400023
...,...
11070,21240568200015
11071,21240568200015
11072,20005355100012
11073,20005355100012


In [71]:
df_keys.to_frame().columns

RangeIndex(start=0, stop=1, step=1)

In [73]:
dfcachee = loading_cache(r"/home/gaspard/Documents/decp-augmente/cache/cache_StockEtablissement_utf8.csv")

In [83]:
pd.concat([df.loc[:, "idTitulaires"], df.loc[:, "acheteur.id"]]).nunique()

11880

In [87]:
df.loc[:, "idTitulaires"].nunique()

8876

In [80]:
df.loc[:, ["idTitulaires", "acheteur.id"]]

,idTitulaires,acheteur.id
0,49994465000013,21500218900011
1,38014942700136,22250001900013
2,43210532800020,18006702700029
3,54210765111679,77605646700587
4,43001432400023,13001537300017
...,...,...
11070,42223879000014,21240568200015
11071,32935942600015,21240568200015
11072,55204499201921,20005355100012
11073,57188001200075,20005355100012


In [75]:
dfcachee

,siren,nic,siret,typeVoieEtablissement,libelleVoieEtablissement,codePostalEtablissement,libelleCommuneEtablissement,codeCommuneEtablissement,activitePrincipaleEtablissement,nomenclatureActivitePrincipaleEtablissement
8110,7350200,49,00735020000049,ALL,DES TILLEULS,04200,SISTERON,04209,43.12B,NAFRev2
13248,15651540,21,01565154000021,RUE,AU BOUCHET,21000,DIJON,21231,43.99C,NAFRev2
15644,16450215,21,01645021500021,RUE,LAVOISIER,21700,NUITS-SAINT-GEORGES,21464,42.11Z,NAFRev2
17422,17051186,25,01705118600025,RUE,DE LA PIECE LEGER,21160,MARSANNAY-LA-COTE,21390,43.12A,NAFRev2
17785,17180043,22,01718004300022,RTE,DE ROUEN,76270,QUIEVRECOURT,76516,81.30Z,NAFRev2
...,...,...,...,...,...,...,...,...,...,...
33420865,998299804,67,99829980400067,RUE,THOMAS EDISON,60180,NOGENT-SUR-OISE,60463,17.22Z,NAFRev2
33421577,998640304,15,99864030400015,RUE,DU GENERAL LECLERC,92130,ISSY-LES-MOULINEAUX,92040,71.12B,NAFRev2
33421719,998724207,27,99872420700027,RUE,DES CAPUCINES,45160,OLIVET,45232,33.20D,NAFRev2
33424515,998823504,31782,99882350431782,RUE,HENRI LEGAY,69100,VILLEURBANNE,69266,78.20Z,NAFRev2


# Fix de error "cannot convert non finite values" dans lec as du cache double

In [93]:
with open(r"/home/gaspard/Documents/decp-augmente/dfcache", "rb") as cache:
    dfcache = pickle.load(cache)

with open(r"/home/gaspard/Documents/decp-augmente/dfSIRET", "rb") as siret:
    dfsiret = pickle.load(siret)

with open(r"/home/gaspard/Documents/decp-augmente/enrich", "rb") as enr:
    enr = pickle.load(enr)

In [94]:
dfcache

,siren,nic,siret,typeVoieEtablissement,libelleVoieEtablissement,codePostalEtablissement,libelleCommuneEtablissement,codeCommuneEtablissement,activitePrincipaleEtablissement,nomenclatureActivitePrincipaleEtablissement
8110,7350200,49,00735020000049,ALL,DES TILLEULS,04200,SISTERON,04209,43.12B,NAFRev2
13248,15651540,21,01565154000021,RUE,AU BOUCHET,21000,DIJON,21231,43.99C,NAFRev2
15644,16450215,21,01645021500021,RUE,LAVOISIER,21700,NUITS-SAINT-GEORGES,21464,42.11Z,NAFRev2
17422,17051186,25,01705118600025,RUE,DE LA PIECE LEGER,21160,MARSANNAY-LA-COTE,21390,43.12A,NAFRev2
17785,17180043,22,01718004300022,RTE,DE ROUEN,76270,QUIEVRECOURT,76516,81.30Z,NAFRev2
...,...,...,...,...,...,...,...,...,...,...
33420865,998299804,67,99829980400067,RUE,THOMAS EDISON,60180,NOGENT-SUR-OISE,60463,17.22Z,NAFRev2
33421577,998640304,15,99864030400015,RUE,DU GENERAL LECLERC,92130,ISSY-LES-MOULINEAUX,92040,71.12B,NAFRev2
33421719,998724207,27,99872420700027,RUE,DES CAPUCINES,45160,OLIVET,45232,33.20D,NAFRev2
33424515,998823504,31782,99882350431782,RUE,HENRI LEGAY,69100,VILLEURBANNE,69266,78.20Z,NAFRev2


In [95]:
dfsiret

,siret,siren,denominationSociale
0,49994465000013,499944650,cuisine pro services
1,38014942700136,380149427,SPIE BATIGNOLLES FONDATIONS SA
2,43210532800020,432105328,PC21
3,54210765111679,542107651,ENGIE
4,43001432400023,430014324,BLANCHISSERIE 2000
...,...,...,...
8870,79504551700013,795045517,A.BINTEIN SARL
8871,68204083701588,682040837,SMAC
8873,32935942600015,329359426,SOGEME
8874,57188001200075,571880012,FMB


In [96]:
enr

,siret,siren,denominationSociale,nic,typeVoieEtablissement,libelleVoieEtablissement,codePostalEtablissement,libelleCommuneEtablissement,codeCommuneEtablissement,activitePrincipaleEtablissement,nomenclatureActivitePrincipaleEtablissement
0,49994465000013,499944650,cuisine pro services,13,<NA>,ZONE ARTISANALE,50290,BREHAL,50076,46.69B,NAFRev2
1,38014942700136,380149427,SPIE BATIGNOLLES FONDATIONS SA,136,AV,DU GENERAL GALLIENI,92000,NANTERRE,92050,43.99D,NAFRev2
2,43210532800020,432105328,PC21,20,ALL,ROLAND GARROS,93360,NEUILLY-PLAISANCE,93049,46.51Z,NAFRev2
3,54210765111679,542107651,ENGIE,11679,RUE,LOUIS BLERIOT,76230,BOIS-GUILLAUME,76108,35.23Z,NAFRev2
4,43001432400023,430014324,BLANCHISSERIE 2000,23,AV,DES TEMPLIERS,13400,AUBAGNE,13005,96.01A,NAFRev2
...,...,...,...,...,...,...,...,...,...,...,...
8628,79504551700013,795045517,A.BINTEIN SARL,13,IMP,DE LA RENAUDIERE,76770,MALAUNAY,76402,49.32Z,NAFRev2
8629,68204083701588,682040837,SMAC,1588,RUE,FANFAN LA TULIPE,92100,BOULOGNE-BILLANCOURT,92012,43.99A,NAFRev2
8630,32935942600015,329359426,SOGEME,15,<NA>,GRAND PONT RTE BORDEAUX,33220,PORT-SAINTE-FOY-ET-PONCHAPT,24335,43.32A,NAFRev2
8631,57188001200075,571880012,FMB,75,RTE,DU CIRCUIT,58490,SAINT-PARIZE-LE-CHATEL,58260,46.39A,NAFRev2


# Opti ENRICHISSEMENT ENTREPRISE

In [249]:
def enrichissement_type_entreprise(df: pd.DataFrame) -> pd.DataFrame:
    """
    Enrichissement des données avec la catégorie de l'entreprise. Utilisation de la base StockUniteLegale de l'Insee

    Retour:
        - pd.DataFrame
    """
    #df = df.astype(conf_glob["enrichissement"]["type_col_enrichissement_siret"], copy=False)
    # Recuperation de la base
    path = r"/home/gaspard/Documents/decp-augmente/data/StockUniteLegale_utf8.csv"
    # La base est volumineuse. Pour "optimiser la mémoire", on va segmenter l'import
    to_add = pd.DataFrame(columns=["siren", "categorieEntreprise"])
    usecols=["siren", "categorieEntreprise", "nicSiegeUniteLegale"]
    dtype={"siren": 'string', "categorieEntreprise": 'string', "nicSiegeUniteLegale": 'string'}
    
    
    
    chunksize = 1000000
    for to_add_chunk in pd.read_csv(
        path,
        chunksize=chunksize,
        usecols=["siren", "categorieEntreprise", "nicSiegeUniteLegale"],
        dtype={"siren": 'string', "categorieEntreprise": 'string', "nicSiegeUniteLegale": 'string'}
    ):
        # On doit creer Siret
        to_add_chunk["nicSiegeUniteLegale"] = to_add_chunk["nicSiegeUniteLegale"].astype(str).str.zfill(5)

        #  À Partir d'ici le siren correspond à siretEtablissement
        #  C'est la même colonne pour optimiser la mémoire
        to_add_chunk["siren"] = to_add_chunk["siren"].astype(str).str\
            .cat(to_add_chunk["nicSiegeUniteLegale"].astype(str), sep='')
        # filtrer only existing siret
        to_add = to_add.append(to_add_chunk[to_add_chunk['siren'].isin(df['siretEtablissement'])])


    to_add.rename(columns={"siren": "siretEtablissement"}, inplace=True)
    # # Jointure sur le Siret entre df et to_add
    df = df.merge(
        to_add[['categorieEntreprise', 'siretEtablissement']], how='left', on='siretEtablissement', copy=False)
    df["categorieEntreprise"] = np.where(df["categorieEntreprise"].isnull(), "NC", df["categorieEntreprise"])
    return df, to_add


In [252]:
df = df_geo.copy()
out_no_cache, to_add  = enrichissement_type_entreprise(df)

/tmp/ipykernel_25995/4039293709.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  to_add = to_add.append(to_add_chunk[to_add_chunk['siren'].isin(df['siretEtablissement'])])
/tmp/ipykernel_25995/4039293709.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  to_add = to_add.append(to_add_chunk[to_add_chunk['siren'].isin(df['siretEtablissement'])])
/tmp/ipykernel_25995/4039293709.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  to_add = to_add.append(to_add_chunk[to_add_chunk['siren'].isin(df['siretEtablissement'])])
/tmp/ipykernel_25995/4039293709.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  to_add = to_add.append(to_add_c

In [258]:
L = ['49994465000013',
 '54210765111679',
 '38012986609569',
 '38012986609569',
 '38012986609569',
 '50503704400631',
 '50503704400631',
 '19570057000037',
 '75240485500013',
 '77572139200030',
 '53300505400010']
print(out_no_cache[out_no_cache.siretEtablissement.isin(L)].siretEtablissement)

0     49994465000013
3     54210765111679
5     38012986609569
6     38012986609569
7     38012986609569
17    50503704400631
18    50503704400631
24    19570057000037
33    75240485500013
34    77572139200030
35    53300505400010
Name: siretEtablissement, dtype: object


In [236]:
path = r"/home/gaspard/Documents/decp-augmente/data/StockUniteLegale_utf8.csv"
    # La base est volumineuse. Pour "optimiser la mémoire", on va segmenter l'import

In [183]:
with open(r"/home/gaspard/Documents/decp-augmente/df_apres_enr_geo", "rb") as file:
    df_geo = pickle.load(file)

In [227]:
def actualiser_cache_entreprise(df_to_analyse, path_to_db, dfcache, columns, dtypes, chunksize=1000000):
    """
    Arguments
    ----------
    df_to_analyse (Series) avec une unique colonne

    Returns
    --------------
    

    La fonction parcourt la bdd Insee par chunk. Pour chaque chunk on regarde si il y a des correspondances de siret entre df_to_analyse et la bdd.
    Si il y a un match, on ajoute alors les lignes de la bdd insee au dataframe cache. Sinon c'est que les siret sont à la fois valide, mais non présent dans le cache.
    On les sépare (on retire ceux trouvé de df_to_analyse) pour pouvoir les mettre dans un second cache.
    """
    for gm_chunk in pd.read_csv(path_to_db, chunksize=chunksize, sep=',', encoding='utf-8', usecols=columns, dtype=dtypes):
        
        #Spécificité de cette fonction, on doit pre process la donnée du chunk pour la comparer
        gm_chunk["nicSiegeUniteLegale"] = gm_chunk["nicSiegeUniteLegale"].astype(str).str.zfill(5)
        gm_chunk["siren"] = gm_chunk["siren"].astype(str).str\
            .cat(gm_chunk["nicSiegeUniteLegale"].astype(str), sep='')
        gm_chunk.rename(columns={"siren":"siretEtablissement"}, inplace= True)
        # Ajouter à df cache les infos qu'il faut
        matching = gm_chunk.loc[gm_chunk.siretEtablissement.isin(df_to_analyse.tolist())].copy() # La copie du dataframe qui match parmis le chunk en cours
        df_to_analyse = df_to_analyse[~df_to_analyse.isin(matching.siretEtablissement.tolist())] 
        dfcache = dfcache.append(matching)

    return dfcache, df_to_analyse

In [177]:
def split_on_column_match(dfSIRET: pd.DataFrame, dfcache: pd.DataFrame, column: str):
    """
    La fonction retourne deux series, la premier correspond aux éléments de dfSIRET n'étant pas dans dfcache (selon la colonne en entrée).
    Le second correspond aux éléments étant dans le cache.
    

    Arguments
    -------------
    dfSIRET : dataframe a analyser
    dfcache : dataframe correspondant au cache
    column : nom de la colonne sur laquelle on split en deux dfSIRET
    
    Returns
    -----------
    Deux series.
    """
    print(dfcache)
    mask_boolean_on_column = dfSIRET.loc[:, str(column)].isin(dfcache.loc[:, str(column)].tolist())
    return dfSIRET.loc[~mask_boolean_on_column, str(column)].copy(), dfSIRET.loc[mask_boolean_on_column, str(column)].copy()

def loading_cache(path_to_cache):
    with open(path_to_cache, 'rb') as df_cache:
        df = pickle.load(df_cache)
    return df
def write_cache(dfcache, path_to_cache):
    with open(path_to_cache, 'wb') as pathcache:
        pickle.dump(dfcache, pathcache)
    return None

In [196]:
with open(r"/home/gaspard/Documents/decp-augmente/cache_notin_legale", "rb") as file:
    dfb = pickle.load(file)
len(dfb)

2693

In [218]:
with open(r"/home/gaspard/Documents/decp-augmente/cache_legale", "rb") as file:
    dfa = pickle.load(file)
dfa

,siren,categorieEntreprise,nicSiegeUniteLegale
116662,05781313100026,ETI,00026
992387,31190273800039,PME,00039
1345365,31712174700024,PME,00024
2157065,32696723900012,PME,00012
3361361,34177311700054,PME,00054
3483137,34305826900030,ETI,00030
4303798,35373501200018,PME,00018
4613040,38014942700136,GE,00136
5024592,38453319600018,PME,00018
5111170,38678011800058,PME,00058


In [169]:
def enrichissement_type_entreprise_cache(df: pd.DataFrame) -> pd.DataFrame:
    """
    Enrichissement des données avec la catégorie de l'entreprise. Utilisation de la base StockUniteLegale de l'Insee

    Retour:
        - pd.DataFrame
    """
    #df = df.astype(conf_glob["enrichissement"]["type_col_enrichissement_siret"], copy=False)
    # Recuperation de la base
    path = r"/home/gaspard/Documents/decp-augmente/data/StockUniteLegale_utf8.csv"
    # La base est volumineuse. Pour "optimiser la mémoire", on va segmenter l'import
    to_add = pd.DataFrame(columns=["siren", "categorieEntreprise"])
    usecols=["siren", "categorieEntreprise", "nicSiegeUniteLegale"]
    dtype={"siren": 'string', "categorieEntreprise": 'string', "nicSiegeUniteLegale": 'string'}
    path_cache = "cache_legale"
    path_cache_not_in_bdd = "cache_notin_legale"
    cache_siren_not_found_exists = os.path.isfile(path_cache_not_in_bdd)
    if cache_siren_not_found_exists:
        list_siret_not_found = loading_cache(path_cache_not_in_bdd)
    else:
        list_siret_not_found = []
    chunksize = 1000000
    mask_siren_valid_not_found = df.siretEtablissement.isin(list_siret_not_found)
    dfSIRET_valide_notfound = df[mask_siren_valid_not_found]
    # On retire les siret valides mais non trouvés lors des précédents passages du df.
    df = df[~mask_siren_valid_not_found]

    cache_exists = os.path.isfile(path_cache)
    if cache_exists:
        dfcache = loading_cache(path_cache)
        dfcache = dfcache.rename(columns={"siren" : "siretEtablissement"})
        series_siren_not_in_cache, series_siren_in_cache = split_on_column_match(df, dfcache, column="siretEtablissement")
        need_refresh_cache = not(series_siren_not_in_cache.empty)

        if need_refresh_cache:
            dfcache, series_siren_valid_but_not_found_in_bdd = actualiser_cache_entreprise(series_siren_not_in_cache, path, dfcache, columns=usecols, dtypes=dtype)
            dfcache = dfcache.drop_duplicates(subset=['siren'], keep='first')
            #Update cache de la lsite des sirets valides mais non trouvés
            list_siret_not_found += series_siren_valid_but_not_found_in_bdd.tolist()

            # Actualise les caches
            write_cache(dfcache, path_cache)
            write_cache(list_siret_not_found, path_cache_not_in_bdd)
    else:
        # crécupérer le dataframe correspondant au cache
        dfcache, series_siren_valid_but_not_found_in_bdd = actualiser_cache_entreprise(df.siretEtablissement, path, dfcache=pd.DataFrame(), columns=usecols, dtypes=dtype)
        print(dfcache, series_siren_valid_but_not_found_in_bdd)
        dfcache = dfcache.drop_duplicates(subset=['siretEtablissement'], keep='first')
        # Créer les cache
        write_cache(dfcache, path_cache)
        write_cache(series_siren_valid_but_not_found_in_bdd.tolist(), path_cache_not_in_bdd)

    to_add.rename(columns={"siren": "siretEtablissement"}, inplace=True)
    # # Jointure sur le Siret entre df et to_add
    df = df.merge(
        dfcache[['categorieEntreprise', 'siretEtablissement']], how='left', on='siretEtablissement', copy=False)
    df["categorieEntreprise"] = np.where(df["categorieEntreprise"].isnull(), "NC", df["categorieEntreprise"])
    del to_add
    return df

In [259]:
df = df_geo.copy()
# Recuperation de la base
path = r"/home/gaspard/Documents/decp-augmente/data/StockUniteLegale_utf8.csv"
# La base est volumineuse. Pour "optimiser la mémoire", on va segmenter l'import
to_add = pd.DataFrame(columns=["siren", "categorieEntreprise"])
usecols=["siren", "categorieEntreprise", "nicSiegeUniteLegale"]
dtype={"siren": 'string', "categorieEntreprise": 'string', "nicSiegeUniteLegale": 'string'}
path_cache = "cache_legale"
path_cache_not_in_bdd = "cache_notin_legale"
cache_siren_not_found_exists = os.path.isfile(path_cache_not_in_bdd)
if cache_siren_not_found_exists:
    list_siret_not_found = loading_cache(path_cache_not_in_bdd)
else:
    list_siret_not_found = []
chunksize = 1000000

mask_siren_valid_not_found = df.siretEtablissement.isin(list_siret_not_found)
dfSIRET_valide_notfound = df[mask_siren_valid_not_found]
# On retire les siret valides mais non trouvés lors des précédents passages du df.
df = df[~mask_siren_valid_not_found]
cache_exists = os.path.isfile(path_cache)
if cache_exists:
    print("cache existe")
    dfcache = loading_cache(path_cache)

    dfcache = dfcache.rename(columns={"siren" : "siretEtablissement"})
    series_siren_not_in_cache, series_siren_in_cache = split_on_column_match(df, dfcache, column="siretEtablissement")
    print(dfcache.head())
    print('eeee \n', series_siren_not_in_cache.head())
    need_refresh_cache = not(series_siren_not_in_cache.empty)
    if need_refresh_cache:
        dfcache, series_siren_valid_but_not_found_in_bdd = actualiser_cache_entreprise(series_siren_not_in_cache, path, dfcache, columns=usecols, dtypes=dtype)
        dfcache = dfcache.rename(columns={"siren" : "siretEtablissement"})
        print("need ", series_siren_valid_but_not_found_in_bdd.head())
        dfcache = dfcache.drop_duplicates(subset=['siretEtablissement'], keep='first')
        #Update cache de la lsite des sirets valides mais non trouvés
        list_siret_not_found += series_siren_valid_but_not_found_in_bdd.tolist()

        # Actualise les caches
        write_cache(dfcache, path_cache)
        write_cache(list_siret_not_found, path_cache_not_in_bdd)
else:
    print('creation')
    # crécupérer le dataframe correspondant au cache
    dfcache, series_siren_valid_but_not_found_in_bdd = actualiser_cache_entreprise(df.siretEtablissement, path, dfcache=pd.DataFrame(), columns=usecols, dtypes=dtype)
    print(dfcache.head(), series_siren_valid_but_not_found_in_bdd.head())
    dfcache = dfcache.drop_duplicates(subset=['siretEtablissement'], keep='first') # La colonne s'appelle encore siren dans le cache
    # Créer les cache
    write_cache(dfcache, path_cache)
    write_cache(series_siren_valid_but_not_found_in_bdd.tolist(), path_cache_not_in_bdd)

dfcache.rename(columns={"siren": "siretEtablissement"}, inplace=True)
# # Jointure sur le Siret entre df et to_add
df = df.merge(
    dfcache[['categorieEntreprise', 'siretEtablissement']], how='left', on='siretEtablissement', copy=False)
df["categorieEntreprise"] = np.where(df["categorieEntreprise"].isnull(), "NC", df["categorieEntreprise"])

df.head()

creation


/tmp/ipykernel_25995/121880461.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfcache = dfcache.append(matching)


cache ent       siretEtablissement categorieEntreprise nicSiegeUniteLegale
7272      00735020000049                 PME               00049
11850     01565154000021                 PME               00021
13567     01645021500021                 PME               00021
14963     01705118600025                 PME               00025
15210     01718004300022                 PME               00022


/tmp/ipykernel_25995/121880461.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfcache = dfcache.append(matching)


cache ent       siretEtablissement categorieEntreprise nicSiegeUniteLegale
7272      00735020000049                 PME               00049
11850     01565154000021                 PME               00021
13567     01645021500021                 PME               00021
14963     01705118600025                 PME               00025
15210     01718004300022                 PME               00022


...

cache ent       siretEtablissement categorieEntreprise nicSiegeUniteLegale
7272      00735020000049                 PME               00049
11850     01565154000021                 PME               00021
13567     01645021500021                 PME               00021
14963     01705118600025                 PME               00025
15210     01718004300022                 PME               00022


/tmp/ipykernel_25995/121880461.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfcache = dfcache.append(matching)


cache ent       siretEtablissement categorieEntreprise nicSiegeUniteLegale
7272      00735020000049                 PME               00049
11850     01565154000021                 PME               00021
13567     01645021500021                 PME               00021
14963     01705118600025                 PME               00025
15210     01718004300022                 PME               00022
cache ent       siretEtablissement categorieEntreprise nicSiegeUniteLegale
7272      00735020000049                 PME               00049
11850     01565154000021                 PME               00021
13567     01645021500021                 PME               00021
14963     01705118600025                 PME               00025
15210     01718004300022                 PME               00022
      siretEtablissement categorieEntreprise nicSiegeUniteLegale
7272      00735020000049                 PME               00049
11850     01565154000021                 PME               00021
13567

/tmp/ipykernel_25995/121880461.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfcache = dfcache.append(matching)


,procedure,source,nature,codeCPV_Original,dureeMois,datePublicationDonnees,id,type,formePrix,dateNotification,...,latitudeCommuneAcheteur,longitudeCommuneAcheteur,superficieCommuneEtablissement,populationCommuneEtablissement,latitudeCommuneEtablissement,longitudeCommuneEtablissement,distanceAcheteurEtablissement,geolocCommuneAcheteur,geolocCommuneEtablissement,categorieEntreprise
0,Procédure adaptée,marches-publics.info,MARCHE,39314000,1,2019-05-21,2018180800100,Marché,Ferme,2018-06-20,...,48.8327939183,-1.5661139044,1315.0,3070.0,48.8971511325,-1.53067522665,8.163603,"48.8327939183,-1.5661139044","48.8971511325,-1.53067522665",NC
1,Appel d'offres ouvert,data.gouv.fr_pes,MARCHE,45210000-2,35,2021-07-12,202130054002,Marché,Révisable,2021-06-15,...,47.2551269927,6.0192810348,1224.0,90722.0,48.8963161116,2.20694242609,459.256127,"47.2551269927,6.0192810348","48.8963161116,2.20694242609",GE
2,Appel d'offres ouvert,data.gouv.fr_aife,ACCORD-CADRE,30213100,48,2021-10-06,20212021F1799800,Marché,<NA>,2021-10-06,...,48.8625262113,2.33630086089,342.0,20755.0,48.8637518036,2.51036788744,19.248164,"48.8625262113,2.33630086089","48.8637518036,2.51036788744",PME
3,Appel d'offres ouvert,data.gouv.fr_aife,ACCORD-CADRE,9310000,15,2020-10-12,20202020F1529600,Marché,<NA>,2020-10-12,...,48.8481450572,2.59605262949,884.0,12847.0,49.4744089013,1.11792062882,177.677307,"48.8481450572,2.59605262949","49.4744089013,1.11792062882",NC
4,Appel d'offres ouvert,data.gouv.fr_aife,ACCORD-CADRE,98310000,48,2021-12-02,2021130017121600,Marché,Révisable,2021-12-02,...,nan,nan,5484.0,45243.0,43.2936269646,5.56325220466,NaN,NaN,"43.2936269646,5.56325220466",PME


# Etudions le fonctionnement de LUHN

In [266]:
pd.DataFrame(data={"idAcheteur": ["12"]})

,idAcheteur
0,12


In [270]:
df_geo = df_geo.append(pd.DataFrame(data={"idAcheteur": ["12"]})).reset_index()
df_geo.idAcheteur

/tmp/ipykernel_25995/3844265175.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_geo = df_geo.append(pd.DataFrame(data={"idAcheteur": ["12"]})).reset_index()


0        21500218900011
1        22250001900013
2        18006702700029
3        77605646700587
4        13001537300017
              ...      
10816    20005355100012
10817    19790023600013
10818                12
10819                12
10820                12
Name: idAcheteur, Length: 10821, dtype: object

In [284]:
df.typeIdentifiantEtablissement

0        SIRET
1        SIRET
2        SIRET
3        SIRET
4        SIRET
         ...  
10816    SIRET
10817    SIRET
10818      NaN
10819      NaN
10820      NaN
Name: typeIdentifiantEtablissement, Length: 10821, dtype: object

In [282]:
df = df_geo.copy()
print('init')
print(df.shape)
df['siren1Acheteur'] = df["idAcheteur"].str[:9]
df_SA = pd.DataFrame(df['siren1Acheteur'])
df_SA = df_SA.drop_duplicates(subset=['siren1Acheteur'], keep='first')
print(df.shape)
print('SA')
print(df_SA.shape)
df_SA['sirenAcheteurValide'] = df_SA['siren1Acheteur'].apply(is_luhn_valid)
df = pd.merge(df, df_SA, how='left', on='siren1Acheteur', copy=False)
print('final')
print(df.shape)
del df['siren1Acheteur']
del df_SA
print('final')
print(df.sirenAcheteurValide)

init
(10821, 66)
(10821, 67)
SA
(2709, 1)
final
(10821, 68)
final
0         True
1         True
2         True
3         True
4         True
         ...  
10816     True
10817     True
10818    False
10819    False
10820    False
Name: sirenAcheteurValide, Length: 10821, dtype: bool


In [272]:
def is_luhn_valid(x: int) -> bool:
    """
    Application de la formule de Luhn à un nombre
    Permet la verification du numero SIREN et Siret d'un acheteur/etablissement

    Retour:
        - bool
    """
    try:
        luhn_corr = [0, 2, 4, 6, 8, 1, 3, 5, 7, 9]
        list_number_in_x = [int(i) for i in list(str(x))]
        l2 = [luhn_corr[i] if (index + 1) % 2 == 0 else i for index, i in enumerate(list_number_in_x[::-1])]
        if sum(l2) % 10 == 0:
            return True
        elif str(x)[:9] == "356000000":  # SIREN de la Poste
            if sum(list_number_in_x) % 5 == 0:
                return True
        return False
    except:
        return False


In [285]:
with open("df_nettoye", "rb") as file:
    df_n = pickle.load(file)

In [287]:
df_n.head()

,procedure,source,uid,nature,codeCPV,dureeMois,datePublicationDonnees,id,_type,formePrix,...,CPV_min,natureObjet,codeDepartementExecution,codeRegionExecution,libelle,libelleRegionExecution,anneeNotification,moisNotification,dureeMoisEstimee,dureeMoisCalculee
0,Procédure adaptée,marches-publics.info,215002189000112018180800100,Marché,39314000,1,2019-05-21,2018180800100,Marché,Ferme,...,39,Fournitures,50,nan,Manche,NaN,2018,06,False,1.0
1,Appel d'offres ouvert,data.gouv.fr_pes,22250001900013202130054002,Marché,45210000-2,35,2021-07-12,202130054002,Marché,Révisable,...,45,Travaux,25,nan,Doubs,NaN,2021,06,False,35.0
2,Appel d'offres ouvert,data.gouv.fr_aife,1800670270002920212021F1799800,Accord-cadre,30213100,48,2021-10-06,20212021F1799800,Marché,NaN,...,30,Fournitures,75,nan,Paris,NaN,2021,10,False,48.0
3,Appel d'offres ouvert,data.gouv.fr_aife,7760564670058720202020F1529600,Accord-cadre,9310000,15,2020-10-12,20202020F1529600,Marché,NaN,...,93,Services,75,nan,Paris,NaN,2020,10,False,15.0
4,Appel d'offres ouvert,data.gouv.fr_aife,130015373000172021130017121600,Accord-cadre,98310000,48,2021-12-02,2021130017121600,Marché,Révisable,...,98,Services,83,nan,Var,NaN,2021,12,False,48.0


In [307]:
with open("df_nettoye", "rb") as file:
    df_nomodif = pickle.load(file)

In [311]:
%%timeit
df_nomodif.source

3.12 µs ± 122 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [313]:
%%timeit
df_nomodif["source"]

1.21 µs ± 6.23 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [314]:
%%timeit
df_nomodif.loc[:, 'source']

12.6 µs ± 123 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [315]:
is_luhn_valid("000000000")

True

In [322]:
with open("df_nettoye", "wb") as file:
    pickle.dump(df_nomodif_mini, file)

In [321]:
df_nomodif_mini = df_nomodif.iloc[0:300000].copy()

In [319]:
df_nomodif_mini

,id,source,uid,_type,objet,codeCPV,dureeMois,dateNotification,datePublicationDonnees,montant,...,CPV_min,natureObjet,codeDepartementExecution,codeRegionExecution,libelle,libelleRegionExecution,anneeNotification,moisNotification,dureeMoisEstimee,dureeMoisCalculee
0,00,atexo-maximilien,00,Marché,Le présent marché public a pour objet l'achat ...,22900000,48,2018-09-13,2018-09-13,20000.0,...,22,Fournitures,92,nan,Hauts-de-Seine,NaN,2018,09,True,2.0
1,0000000000000000,data.gouv.fr_aife,00000000000000,Contrat de concession,<NA>,00000000,5,NaN,2019-06-12,3351700.0,...,00,Fournitures,68,nan,Haut-Rhin,NaN,nan,NaN,False,5.0
2,2018-s_SxZ6PAr00,data.gouv.fr_aife,000000000000002018-s_SxZ6PAr00,Marché,CONSTRUCTION D'UN RESTAURANT SCOLAIRE A L'XXXX...,45212000,7,2018-01-08,2020-01-10,57358.0,...,45,Travaux,93,nan,Seine-Saint-Denis,NaN,2018,01,False,7.0
3,20183WC4e9_iEw00,data.gouv.fr_aife,0000000000000020183WC4e9_iEw00,Marché,Maintenance progiciel Symphony,72267000,12,2018-10-16,2019-04-02,26250.0,...,72,Services,94,nan,Val-de-Marne,NaN,2018,10,False,12.0
4,20184CMDlZYWaE00,data.gouv.fr_aife,0000000000000020184CMDlZYWaE00,Marché,ETUDE DE FAISABILITE QUANT A LA STRUCTURATION ...,79314000,6,NaN,2019-01-04,119151.0,...,79,Services,nan,nan,NaN,NaN,nan,NaN,False,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2021100015580700,data.gouv.fr_aife,130001704005892021100015580700,Marché,RN136 : Création de 2 Voies d’Entrecroisement ...,71310000,27,2021-09-01,2021-09-01,425900.0,...,71,Services,35,nan,Ille-et-Vilaine,NaN,2021,09,False,27.0
9996,2021100015642300,data.gouv.fr_aife,130001704005892021100015642300,Marché,mission de maîtrise d’œuvre relative à l’opéra...,71312000,39,2021-09-09,2021-09-09,52190.0,...,71,Services,29,nan,Finistère,NaN,2021,09,False,39.0
9997,2021100015755300,data.gouv.fr_aife,130001704005892021100015755300,Marché,Mission de maîtrise d’œuvre relative au progra...,71312000,38,2021-10-29,2021-10-29,34990.0,...,71,Services,22,nan,Côtes-d'Armor,NaN,2021,10,True,1.0
9998,2021100015767300,data.gouv.fr_aife,130001704005892021100015767300,Marché,Les prestations attendues comprennent l’approp...,71312000,38,2021-10-28,2021-10-28,34350.0,...,71,Services,35,nan,Ille-et-Vilaine,NaN,2021,10,True,1.0


In [316]:
df_nomodif.siren

AttributeError: 'DataFrame' object has no attribute 'siren'

In [288]:
with open("df_augmente_avec_luhn", "rb") as file:
    df_luhn = pickle.load(file)

In [304]:
df_nomodif[df_nomodif.booleanModification==1].objetModification

20                     CONSTRUCTION PISTE CYCLABLE - RD225
27                                            AVENANT 1 TF
28                                            AVENANT 1 TF
47       Affermissement tranche no2 d'un montant de 179...
123                           Affermissement de la tranche
                               ...                        
10906                                 AVENANT 1 PLUS VALUE
10934                                        MISE AU POINT
10953                                     REV ROTH SAD J.C
10957                                            AVENANT 1
10989    Avenant comptable n°3 Prise en compte globalit...
Name: objetModification, Length: 513, dtype: object

In [295]:
df_luhn.siretEtablissement

0       38014942700136
1       43210532800020
2       43001432400023
3       41317505000058
4       41317505000058
             ...      
8104    37964844700033
8105    79504551700013
8106    32935942600015
8107    57188001200075
8108    34946996500025
Name: siretEtablissement, Length: 8109, dtype: object

In [291]:
df_luhn['r'] = False

In [ ]:
def apply_luhn(df: pd.DataFrame) -> pd.DataFrame:
    """
    Application de la formule de Luhn sur les siren/siret

    Retour:
        - pd.DataFramewith open("df_nettoye", "rb") as file:
    df_n = pickle.load(file)
    """
    logger.info("Début du traitement: Vérification Siren/Siret par formule de Luhn")
    # Application sur les siren des Acheteur
    df['siren1Acheteur'] = df["idAcheteur"].str[:9]
    df_SA = pd.DataFrame(df['siren1Acheteur'])
    df_SA = df_SA.drop_duplicates(subset=['siren1Acheteur'], keep='first')
    df_SA['sirenAcheteurValide'] = df_SA['siren1Acheteur'].apply(is_luhn_valid)
    df = pd.merge(df, df_SA, how='left', on='siren1Acheteur', copy=False)
    logger.info("Nombre de Siren Acheteur jugé invalide:{}".format(len(df) - sum(df.sirenAcheteurValide)))
    del df['siren1Acheteur']
    del df_SA
    # Application sur les siren des établissements
    df['siren2Etablissement'] = df.sirenEtablissement.str[:]
    df_SE = pd.DataFrame(df['siren2Etablissement'])
    df_SE = df_SE.drop_duplicates(subset=['siren2Etablissement'], keep='first')
    df_SE['sirenEtablissementValide'] = df_SE['siren2Etablissement'].apply(is_luhn_valid)
    df = pd.merge(df, df_SE, how='left', on='siren2Etablissement', copy=False)
    logger.info("Nombre de Siren Etablissement jugé invalide:{}".format(len(df) - sum(df.sirenEtablissementValide)))
    del df['siren2Etablissement']
    del df_SE
    # Application sur les siret des établissements
    df['siret2Etablissement'] = df.siretEtablissement.str[:]
    df_SE2 = pd.DataFrame(df['siret2Etablissement'])
    df_SE2 = df_SE2.drop_duplicates(subset=['siret2Etablissement'], keep='first')
    df_SE2['siretEtablissementValide'] = df_SE2['siret2Etablissement'].apply(is_luhn_valid)
    # Merge avec le df principal
    df = pd.merge(df, df_SE2, how='left', on='siret2Etablissement', copy=False)
    logger.info("Nombre de Siret Etablissement jugé invalide:{}".format(len(df) - sum(df.siretEtablissementValide)))
    del df["siret2Etablissement"]
    #del df_SE2enrichissement_siret
    del df_SE2

    # On rectifie pour les codes non-siret
    df.siretEtablissementValide = np.where(
        (df.typeIdentifiantEtablissement != 'SIRET'),
        "Non valable",
        df.siretEtablissementValide)
    return df